# In this notebook we will continue our project and start creating our ML model, we will start by importing our clean data blocks (In Progress)

In [24]:
#importing our datasets from our ETL / EDA file
import import_ipynb
import tempe_traffic_EDA_ETL
from tempe_traffic_EDA_ETL import fatal_frame
from tempe_traffic_EDA_ETL import nonfatal_frame
from tempe_traffic_EDA_ETL import fatal_drv_info 
from tempe_traffic_EDA_ETL import non_fatal_drv_info 
from tempe_traffic_EDA_ETL import time_location_fatal 
from tempe_traffic_EDA_ETL import time_location_non_fatal 

In [25]:
fatal_drv_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Incidentid       99 non-null     int64  
 1   Age_Drv1         99 non-null     float64
 2   Gender_Drv1      99 non-null     object 
 3   Violation1_Drv1  99 non-null     object 
 4   AlcoholUse_Drv1  99 non-null     object 
 5   DrugUse_Drv1     99 non-null     object 
 6   Age_Drv2         80 non-null     float64
 7   Gender_Drv2      80 non-null     object 
 8   Violation1_Drv2  80 non-null     object 
 9   AlcoholUse_Drv2  80 non-null     object 
 10  DrugUse_Drv2     80 non-null     object 
 11  Totalfatalities  99 non-null     int64  
 12  Totalinjuries    99 non-null     int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 10.2+ KB


In [26]:
non_fatal_drv_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35729 entries, 1 to 39817
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Incidentid       35729 non-null  int64  
 1   Age_Drv1         35729 non-null  float64
 2   Gender_Drv1      35729 non-null  object 
 3   Violation1_Drv1  35729 non-null  object 
 4   AlcoholUse_Drv1  35729 non-null  object 
 5   DrugUse_Drv1     35729 non-null  object 
 6   Age_Drv2         35729 non-null  float64
 7   Gender_Drv2      35729 non-null  object 
 8   Violation1_Drv2  35729 non-null  object 
 9   AlcoholUse_Drv2  35729 non-null  object 
 10  DrugUse_Drv2     35729 non-null  object 
 11  Injuryseverity   35729 non-null  object 
 12  Totalinjuries    35729 non-null  int64  
dtypes: float64(2), int64(2), object(9)
memory usage: 3.8+ MB


In [27]:
time_location_fatal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Incidentid        99 non-null     int64  
 1   Year              99 non-null     int64  
 2   DateTime          99 non-null     object 
 3   StreetName        99 non-null     object 
 4   CrossStreet       99 non-null     object 
 5   Latitude          99 non-null     float64
 6   Longitude         99 non-null     float64
 7   Lightcondition    99 non-null     object 
 8   Weather           99 non-null     object 
 9   SurfaceCondition  99 non-null     object 
dtypes: float64(2), int64(2), object(6)
memory usage: 7.9+ KB


In [28]:
time_location_non_fatal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39818 entries, 0 to 39817
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Incidentid        39818 non-null  int64  
 1   Year              39818 non-null  int64  
 2   DateTime          39818 non-null  object 
 3   StreetName        39799 non-null  object 
 4   CrossStreet       39777 non-null  object 
 5   Latitude          39680 non-null  float64
 6   Longitude         39680 non-null  float64
 7   Lightcondition    39818 non-null  object 
 8   Weather           39818 non-null  object 
 9   SurfaceCondition  39818 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 3.0+ MB
